In [1]:
import numpy as np
import pandas as pd
from pyscf import gto, scf
import ext 
from pySCF_ext import *

/Users/quantum/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import sopy as sp

In [3]:
def D(q, method):
    return pd.concat([Run,pd.Series({'method':method, 'dist':(q.dist(Sum1)).numpy(), 'canons':int(len(q)), 'ld2':q.ld2().numpy(), 'ld1':q.ld1().numpy(),'norm':q.n().numpy()})])

In [4]:

roll, pitch, yaw = 0*np.radians(np.pi/4), 0*np.radians(np.pi/2), 0*np.radians(-np.pi/4)

Run = pd.Series([ roll, pitch, yaw] , index = ['roll','pitch','yaw'])
Run

roll     0.0
pitch    0.0
yaw     -0.0
dtype: float64

In [5]:
mol = gto.Mole()

if False:
    mol.build( atom = '''H 0 0 1;  ''', basis ='STO-3G',spin= 1)

    ne = 1
    canons = 1

    filename = 'H'
if True:

    
    mol.build( atom = f'''H {rotation_matrix_from_euler(roll, pitch, yaw, [2,2,2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [2,-2,-2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [-2,2,-2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [-2,-2,2])} ''', basis ='STO-3G',spin= 0)
    
    filename = 'H4'

    ne = 4
    canons = ne // 2
    
##

if False:
    coord =   """6.290730e-01,6.290730e-01,6.290730e-01
      6.290730e-01,-6.290730e-01 ,-6.290730e-01
     -6.290730e-01, 6.290730e-01 ,-6.290730e-01
     -6.290730e-01,-6.290730e-01 , 6.290730e-01
      3.629073e+00, 6.290730e-01 , 6.290730e-01
      3.629073e+00,-6.290730e-01 ,-6.290730e-01
      2.370927e+00, 6.290730e-01 ,-6.290730e-01
      2.370927e+00,-6.290730e-01 , 6.290730e-01
      6.290730e-01, 3.629073e+00 , 6.290730e-01
     6.290730e-01,  2.370927e+00 ,-6.290730e-01
     -6.290730e-01,  3.629073e+00 ,-6.290730e-01
     -6.290730e-01,  2.370927e+00 , 6.290730e-01
     6.290730e-01,  6.290730e-01 , 3.629073e+00
     6.290730e-01, -6.290730e-01 , 2.370927e+00
     -6.290730e-01,  6.290730e-01,  2.370927e+00
     -6.290730e-01, -6.290730e-01,  3.629073e+00"""
    coords = ''
    for line in coord.split('\n'):
        x,y,z = map(float,line.replace(' ','').split(','))
        coords += f' H {rotation_matrix_from_euler(roll, pitch, yaw, [x,y,z])} ; '
    mol.build( atom = coords, basis ='STO-3G',spin= 0)
    ne = 16
    filename = 'H16'
    canons = ne *4
Run['filename'] = filename

In [6]:
# Select state 
ne = mol.nelectron//2
Run['ne']     = ne 
ne


2

In [7]:
## RHF ##
##     ##
mf = scf.RHF(mol).run()
orb = mf.mo_coeff ##First Level##
if len(np.shape(orb))< 3:
    orb = [orb]
np.shape(orb)

SCF not converged.
SCF energy = -0.504207988317219


(1, 4, 4)

In [8]:
box_size = 50.0
L        = 500
Run['box_size'] = box_size
Run['L']        = L
lattices = 3*[ np.linspace( -box_size/2,box_size/2, L ) ]

Sum1 = get_orbital(mol, orb[0], ne, lattices=lattices)

In [9]:
q = Sum1.fibonacci(  partition = canons, iterate = 10,total_iterate = 100 ,alpha = 1e-6)
q3 = Sum1.boost().decompose(  partition = canons, iterate = 10 ,alpha = 1e-6).unboost()
q2 = Sum1.boost().fibonacci(  partition = canons, iterate = 10,total_iterate = 100 ,alpha = 1e-6).unboost()

In [10]:
Tum1 = { 0: Sum1[0], 1: Sum1[1], 2: Sum1[2], 3: Sum1[3] } 
lattices_dict = { 1:lattices[0], 2:lattices[1], 3:lattices[2]}

In [14]:
tl = ext.reduce(ext.image(Tum1), canons )

tl1 = sp.vector().transpose(tl, lattices_dict)
D(tl1,'reduce')

roll             0.0
pitch            0.0
yaw             -0.0
filename          H4
ne                 2
box_size        50.0
L                500
method        reduce
dist        0.138765
canons             2
ld2         1.414214
ld1              2.0
norm        1.003401
dtype: object

In [15]:
(pd.DataFrame([D(Sum1,'Orbital'),D(q,'fibonacci'),D(q2,'boost_fibonacci'),D(tl1,'reduce')]))

,roll,pitch,yaw,filename,ne,box_size,L,method,dist,canons,ld2,ld1,norm
0,0.0,0.0,-0.0,H4,2,50.0,500,Orbital,0.000000,12,0.712802,1.604461,1.012951
1,0.0,0.0,-0.0,H4,2,50.0,500,fibonacci,0.140041,2,0.996011,1.408571,1.001325
2,0.0,0.0,-0.0,H4,2,50.0,500,boost_fibonacci,0.140058,2,0.995996,1.408549,1.001292
3,0.0,0.0,-0.0,H4,2,50.0,500,reduce,0.138765,2,1.414214,2.000000,1.003401
